In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List, Any, Tuple, Optional
from tqdm.auto import tqdm
import plotly.graph_objects as go

In [2]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        # Print CUDA details
        print(f"CUDA Device: {torch.cuda.get_device_name()}")
        print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated()/1024**2:.2f}MB")
        print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved()/1024**2:.2f}MB")
        return device
    else:
        return torch.device("cpu")

In [3]:
DEVICE = get_device()

In [4]:
def load_model():
    dtype = torch.bfloat16
    
    tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
    model = AutoModelForCausalLM.from_pretrained(
        "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
        torch_dtype=dtype,
        low_cpu_mem_usage=True,  # Optimize memory usage during loading
    )
    model.to(DEVICE)
    model.eval()
    return tokenizer, model

# Load the model and tokenizer
tokenizer, model = load_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [6]:
def prepare_prompt(prompt: str, tokenizer) -> str:
    """Format the prompt using the chat template if available"""
    if hasattr(tokenizer, 'apply_chat_template'):
        messages = [{"role": "user", "content": prompt}]
        formatted_prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        return formatted_prompt
    return prompt

In [7]:
def tokenize_prompt(prompt: str, tokenizer, device=None):
    """Tokenize a prompt and prepare model inputs"""
    # Get appropriate device
    if device is None:
        device = get_device()
        
    # Format prompt
    formatted_prompt = prepare_prompt(prompt, tokenizer)
    
    # Tokenize
    tokenized_prompt = tokenizer(formatted_prompt, return_tensors="pt")
    
    # Move to device
    tokenized_prompt = {k: v.to(device) for k, v in tokenized_prompt.items()}
    
    print(f"Prompt: {formatted_prompt}")
    
    return tokenized_prompt, formatted_prompt

In [8]:
def get_residual_stream(model, 
                        tokenizer, 
                        tokenized_prompt, 
                        max_new_tokens=20, 
                        layer_indices=None, 
                        save_path=None):
    """
    Get and store the residual stream from the model for input tokens and generated tokens.
    Optimized to write activations to disk incrementally to reduce memory usage.
    
    Args:
        model: The transformer model (LlamaForCausalLM)
        tokenizer: The tokenizer associated with the model
        tokenized_prompt: Dictionary containing input_ids and attention_mask tensors
        max_new_tokens: Maximum number of new tokens to generate (not including input tokens)
        layer_indices: Optional list of specific layer indices to capture. If None, capture all layers.
        save_path: Optional path to save the residual streams (as .pt file)
    
    Returns:
        tuple: (all_residual_streams, generated_text, token_ids)
            - all_residual_streams: Dict mapping from layer index to tensors of shape 
              [batch_size, seq_len, hidden_size] representing the residual stream at each layer
            - generated_text: The text including the generated tokens
            - token_ids: The token IDs including the generated tokens
    """
    import os
    import tempfile
    import torch
    from pathlib import Path
    from tqdm.auto import tqdm
    
    batch_size = tokenized_prompt["input_ids"].shape[0]
    device = tokenized_prompt["input_ids"].device
    input_length = tokenized_prompt["input_ids"].shape[1]
    
    # Create a temporary directory for storing activations
    temp_dir = tempfile.mkdtemp(prefix="residual_stream_")
    print(f"Saving activations to temporary directory: {temp_dir}")
    
    # Track which tokens we've processed
    current_token_pos = 0
    
    # Information about sequence length for each step
    seq_length_info = []
    
    # Register hooks to capture residual streams
    hooks = []
    
    # Function to determine if we should capture a specific layer
    def should_capture_layer(idx):
        if layer_indices is None:
            return True
        return idx in layer_indices
    
    # Define hook function to capture and save residual stream directly to disk
    def get_activation(name, layer_idx):
        def hook(module, input, output):
            nonlocal current_token_pos
            
            # For LLaMA models, the residual stream is the input to the layer
            # Get the activation (residual stream)
            activation = input[0].detach().cpu()
            
            # Save the current sequence length
            if len(seq_length_info) <= current_token_pos:
                seq_length_info.append(activation.shape[1])
            
            # Create filename for this activation
            filename = f"layer_{layer_idx}_{name}_token_{current_token_pos}.pt"
            file_path = os.path.join(temp_dir, filename)
            
            # Save activation to disk
            torch.save(activation, file_path)
        return hook
    
    # Register hooks for each transformer layer
    for i, layer in enumerate(model.model.layers):
        if should_capture_layer(i):
            # Capture the input to the input_layernorm which is the residual stream
            hook = layer.input_layernorm.register_forward_hook(get_activation("input_layernorm", i))
            hooks.append(hook)
    
    # Also capture the final norm layer (after the last transformer layer)
    final_layer_idx = len(model.model.layers)
    final_hook = model.model.norm.register_forward_hook(
        get_activation("final_norm", final_layer_idx)
    )
    hooks.append(final_hook)
    
    # Clone the tokenized_prompt to avoid modifying the original
    generation_inputs = {
        "input_ids": tokenized_prompt["input_ids"].clone(),
        "attention_mask": tokenized_prompt["attention_mask"].clone(),
    }
    
    # Custom generation function to handle incrementing token position
    generated_ids = None
    
    try:
        # Save metadata about the generation
        metadata = {
            'input_length': input_length,
            'max_new_tokens': max_new_tokens,
            'layer_indices': layer_indices or list(range(len(model.model.layers) + 1)),  # +1 for final norm
        }
        torch.save(metadata, os.path.join(temp_dir, "metadata.pt"))
        
        # Print information about captured layers
        layer_indices_to_capture = metadata['layer_indices']
        print(f"Capturing {len(layer_indices_to_capture)} layers: {layer_indices_to_capture}")
        
        # We'll use our own token-by-token generation to have more control
        with torch.no_grad():
            # First, process the input sequence
            input_ids = generation_inputs["input_ids"]
            attention_mask = generation_inputs["attention_mask"]
            
            # Show progress info for initial processing
            print("Processing input sequence...")
            # Initial forward pass with the input sequence
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                return_dict=True
            )
            
            print(f"Input sequence processed ({input_length} tokens)")
            
            # Increment token position after processing the input sequence
            current_token_pos += 1
            
            # Continue generating tokens one by one
            generated_sequence = input_ids.clone()
            
            # Initialize progress bar for token generation
            pbar = tqdm(total=max_new_tokens, desc="Generating tokens")
            
            for i in range(max_new_tokens):
                # Get next token prediction
                next_token_logits = outputs.logits[:, -1, :]
                next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
                
                # Append new token to sequence
                generated_sequence = torch.cat([generated_sequence, next_token_id], dim=-1)
                
                # Update progress bar with token info
                token_str = tokenizer.decode(next_token_id[0], skip_special_tokens=False)
                pbar.set_postfix({"token": token_str})
                pbar.update(1)
                
                # Check if we've generated an EOS token
                if next_token_id.item() == tokenizer.eos_token_id:
                    break
                
                # Forward pass with the updated sequence
                # We only need to process the new token, using KV cache for efficiency
                outputs = model(
                    input_ids=next_token_id,
                    attention_mask=torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=1),
                    use_cache=True,
                    past_key_values=outputs.past_key_values,
                    return_dict=True
                )
                
                # Update attention mask for next iteration
                attention_mask = torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=1)
                
                # Increment token position after processing a new token
                current_token_pos += 1
                
            # Close the progress bar
            pbar.close()
            
            # Store the final generated sequence
            generated_ids = generated_sequence
    
    except Exception as e:
        print(f"Error during generation: {e}")
        # Remove hooks
        for hook in hooks:
            hook.remove()
        raise
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    # Save sequence length information
    torch.save(seq_length_info, os.path.join(temp_dir, "seq_length_info.pt"))
    
    # Decode the generated text
    try:
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        # Print a preview of the generated text
        preview = generated_text.replace('\n', '\\n')
        if len(preview) > 100:
            preview = preview[:97] + '...'
        input_tokens = input_length
        output_tokens = generated_ids.shape[1]
        new_tokens = output_tokens - input_tokens
        print(f"Generated text: {preview}")
        print(f"Input tokens: {input_tokens}, New tokens: {new_tokens}, Total tokens: {output_tokens}")
    except Exception as e:
        print(f"Error decoding generated tokens: {e}")
        generated_text = f"[Error decoding tokens: {generated_ids[0]}]"
    
    # Store token ids for reference
    token_ids = generated_ids.detach().cpu()
    torch.save(token_ids, os.path.join(temp_dir, "token_ids.pt"))
    
    # Load the individual files and combine them into the return format
    all_residual_streams = {}
    for layer_idx in metadata['layer_indices']:
        all_residual_streams[layer_idx] = []
        
        # Load all token files for this layer
        for token_pos in range(len(seq_length_info)):
            # Try loading input_layernorm activation
            try:
                filename = f"layer_{layer_idx}_input_layernorm_token_{token_pos}.pt"
                file_path = os.path.join(temp_dir, filename)
                if os.path.exists(file_path):
                    activation = torch.load(file_path)
                    all_residual_streams[layer_idx].append(activation)
                    continue
            except Exception as e:
                pass
            
            # Try loading final_norm activation
            try:
                filename = f"layer_{layer_idx}_final_norm_token_{token_pos}.pt"
                file_path = os.path.join(temp_dir, filename)
                if os.path.exists(file_path):
                    activation = torch.load(file_path)
                    all_residual_streams[layer_idx].append(activation)
                    continue
            except Exception as e:
                pass
        
        # Concatenate activations for this layer
        if all_residual_streams[layer_idx]:
            all_residual_streams[layer_idx] = torch.cat(all_residual_streams[layer_idx], dim=1)
        else:
            print(f"Warning: No activations found for layer {layer_idx}")
            all_residual_streams.pop(layer_idx)
    
    # Save to file if requested
    if save_path is not None:
        save_data = {
            'residual_streams': all_residual_streams,
            'generated_text': generated_text,
            'token_ids': token_ids
        }
        try:
            # Add file extension if not provided
            if not save_path.endswith('.pt'):
                save_path = f"{save_path}.pt"
            
            # Make sure directory exists
            os.makedirs(os.path.dirname(save_path) if os.path.dirname(save_path) else '.', exist_ok=True)
            
            torch.save(save_data, save_path)
            print(f"Residual stream data saved to {save_path} \n-----\n")
        except Exception as e:
            print(f"Error saving residual stream data: {e}")
    
    # Clean up temporary files if requested (can be uncommented if you want to auto-delete temp files)
    # import shutil
    # shutil.rmtree(temp_dir)
    # print(f"Temporary files cleaned up from {temp_dir}")
    
    return all_residual_streams, generated_text, token_ids

In [9]:


def load_residual_stream(load_path):
    """
    Load previously saved residual stream data from a file.
    
    Args:
        load_path: Path to the saved residual stream file (.pt)
        
    Returns:
        tuple: (all_residual_streams, generated_text, token_ids)
    """
    data = torch.load(load_path)
    return data['residual_streams'], data['generated_text'], data['token_ids']

In [10]:
def patch_res_stream(model, 
                    tokenizer, 
                    clean_path, 
                    corrupt_path, 
                    patch_layers, 
                    num_tokens_to_patch=None, 
                    target_token="<think>", 
                    save_path=None, 
                    device=None, 
                    max_new_tokens=100):
    """
    Patch residual streams from corrupted prompt into the clean prompt at specified layers and tokens,
    supporting autoregressive generation with improved progress tracking.
    
    Args:
        model: The transformer model (LlamaForCausalLM)
        tokenizer: The tokenizer associated with the model
        clean_path: Path to the saved clean prompt residual stream file (.pt)
        corrupt_path: Path to the saved corrupted prompt residual stream file (.pt)
        patch_layers: List of layer indices to patch (e.g., [5, 6, 7] or range(32))
        num_tokens_to_patch: Number of tokens to patch after the target token.
                           If None, will patch from the target token to the end.
        target_token: Token after which to start patching (default: "<think>")
        save_path: Optional path to save the patched results (.pt)
        device: Device to perform computation on
        max_new_tokens: Maximum number of new tokens to generate
    
    Returns:
        tuple: (patched_text, original_clean_text, original_corrupt_text, patched_token_ids)
    """
    import os
    import torch
    from tqdm.auto import tqdm
    
    # Set device if not provided
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load the saved residual streams
    print(f"Loading residual streams from {clean_path} and {corrupt_path}...")
    clean_streams, clean_text, clean_ids = load_residual_stream(clean_path)
    corrupt_streams, corrupt_text, corrupt_ids = load_residual_stream(corrupt_path)
    
    # Ensure both clean_ids and corrupt_ids are on CPU for initial processing
    clean_ids = clean_ids.cpu()
    corrupt_ids = corrupt_ids.cpu()
    
    print(f"Clean text preview: {clean_text[:100]}...")
    print(f"Corrupt text preview: {corrupt_text[:100]}...")
    
    # Find the target token index in the clean prompt
    if target_token:
        # Check if target token is a string or a token ID
        if isinstance(target_token, str):
            # Find the token ID for the target token
            target_token_ids = tokenizer.encode(target_token, add_special_tokens=False)
            if len(target_token_ids) != 1:
                print(f"Warning: Target token '{target_token}' encoded to {len(target_token_ids)} tokens: {target_token_ids}")
            target_token_id = target_token_ids[0]
        else:
            target_token_id = target_token
            
        # Find the position of the target token in the clean_ids
        target_positions = (clean_ids[0] == target_token_id).nonzero(as_tuple=True)[0]
        if len(target_positions) == 0:
            print(f"Target token '{target_token}' not found in clean_ids. Will patch from the beginning.")
            patch_start_idx = 0
        else:
            # Take the first occurrence of the target token
            patch_start_idx = target_positions[0].item()
            target_token_text = tokenizer.decode(target_token_id)
            print(f"Found target token '{target_token_text}' (ID: {target_token_id}) at position {patch_start_idx}")
    else:
        # If no target token specified, start patching from the beginning
        patch_start_idx = 0
    
    # Set the patch range
    if num_tokens_to_patch is None:
        # From target token to the end
        patch_end_idx = min(clean_ids.shape[1], corrupt_ids.shape[1])
    else:
        # Patch specified number of tokens after the target token
        patch_end_idx = min(patch_start_idx + num_tokens_to_patch, clean_ids.shape[1], corrupt_ids.shape[1])
    
    patch_tokens_range = (patch_start_idx, patch_end_idx)
    
    print(f"Will patch tokens from position {patch_start_idx} to {patch_end_idx} ({patch_end_idx - patch_start_idx} tokens)")
    print(f"Will patch layers: {patch_layers}")
    
    # Validate layers
    max_layer = max(clean_streams.keys())
    patch_layers = [layer for layer in patch_layers if layer in clean_streams and layer in corrupt_streams]
    if not patch_layers:
        raise ValueError(f"No valid layers to patch. Layers must be in range 0-{max_layer}")
    
    # Preload corrupted residual streams into device memory for faster access during generation
    # Only preload the layers and tokens we need to patch
    corrupted_cache = {}
    
    print("Preloading corrupted residual streams to device...")
    for layer_idx in tqdm(patch_layers, desc="Preloading layers"):
        # Get the corrupted streams for this layer, but only for tokens we'll patch
        corrupted_cache[layer_idx] = corrupt_streams[layer_idx][0, patch_start_idx:patch_end_idx, :].to(device)
    
    # Create patched inputs based on clean prompt
    input_ids = clean_ids[0, :patch_start_idx].unsqueeze(0).to(device)
    attention_mask = torch.ones_like(input_ids).to(device)
    
    patched_inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
    }
    
    # Set up patching state for autoregressive generation
    # Position relative to the start of the patch range (0 = first token to patch)
    patching_pos = 0
    # Absolute position in the sequence
    current_token_pos = patch_start_idx
    
    # Set up patching progress tracking
    expected_tokens_to_patch = min(patch_end_idx - patch_start_idx, max_new_tokens)
    progress_bar = tqdm(total=expected_tokens_to_patch, desc="Patching tokens", ncols=100)
    
    # Create a hook to patch each token during autoregressive generation
    hooks = []
    
    # Dictionary to track patches applied
    patch_stats = {
        "total_tokens": 0,
        "patched_tokens": 0,
        "patched_per_layer": {layer: 0 for layer in patch_layers}
    }
    
    def patch_hook(layer_idx):
        def hook(module, inputs):
            nonlocal patching_pos, current_token_pos
            
            # Get residual stream
            res_stream = inputs[0]
            batch_size, seq_len, hidden_dim = res_stream.shape
            
            # During autoregressive generation, we'll get one token at a time
            # For the initial pass with multiple tokens, patch only the target position and beyond
            if seq_len > 1:
                # This is the initial forward pass with all input tokens
                # We don't patch anything in the input prefix (before target token)
                return inputs
            else:
                # This is autoregressive generation with one token at a time
                # Only update position tracking in the first layer to avoid duplicate counting
                if layer_idx == patch_layers[0]:
                    # Move to the next patching position
                    patching_pos += 1
                    current_token_pos += 1
                    
                    # Update progress bar only once per token (in first layer)
                    if patching_pos <= expected_tokens_to_patch:
                        progress_bar.update(1)
                        patch_stats["total_tokens"] += 1
                
                # Check if this position should be patched
                if patching_pos > 0 and patching_pos <= (patch_end_idx - patch_start_idx):
                    # Get the corresponding token from the preloaded corrupted cache
                    # The position in the cache is (patching_pos - 1) because patching_pos starts at 1
                    to_patch = corrupted_cache[layer_idx][patching_pos - 1, :]
                    
                    # Replace the residual stream for this token with the corrupted version
                    res_stream[0, 0, :] = to_patch
                    
                    # Track stats
                    if layer_idx == patch_layers[0]:
                        patch_stats["patched_tokens"] += 1
                    patch_stats["patched_per_layer"][layer_idx] += 1
                    
                    # Periodically log progress (less verbose)
                    if patching_pos % 10 == 0 and layer_idx == patch_layers[0]:
                        token_text = tokenizer.decode(input_ids[0, -1].item())
                        current_token = token_text.replace('\n', '\\n')
                        progress_bar.set_postfix({"current_token": current_token, "pos": current_token_pos})
            
            # Return potentially modified inputs
            return (res_stream,) + inputs[1:] if len(inputs) > 1 else (res_stream,)
        
        return hook
    
    # Register forward pre-hooks for each layer to patch
    for layer_idx in patch_layers:
        hook = model.model.layers[layer_idx].input_layernorm.register_forward_pre_hook(
            patch_hook(layer_idx)
        )
        hooks.append(hook)
    
    # Generate text with patched residual streams
    try:
        with torch.no_grad():
            # Generate text with the patched residual stream
            generation_output = model.generate(
                **patched_inputs,
                max_new_tokens=max_new_tokens,
                return_dict_in_generate=True,
                output_scores=True,
                pad_token_id=tokenizer.eos_token_id
            )
            patched_ids = generation_output.sequences
    except Exception as e:
        print(f"Error during generation with patched residual stream: {e}")
        for hook in hooks:
            hook.remove()
        raise
    finally:
        # Remove all hooks
        for hook in hooks:
            hook.remove()
        
        # Close progress bar
        progress_bar.close()
    
    # Decode the patched generation
    patched_text = tokenizer.decode(patched_ids[0], skip_special_tokens=True)
    
    # Print summary
    print("\n--- Patching Results ---")
    print(f"Patched layers: {patch_layers}")
    print(f"Patched token range: {patch_tokens_range}")
    print(f"Total tokens processed: {patch_stats['total_tokens']}")
    print(f"Total tokens patched: {patch_stats['patched_tokens']}")
    print(f"Tokens patched per layer: {patch_stats['patched_per_layer']}")
    print(f"Generation length: {patched_ids.shape[1] - input_ids.shape[1]} new tokens")
    
    print("\nOriginal clean output: ")
    print(clean_text[:200] + "..." if len(clean_text) > 200 else clean_text)
    print("\nOriginal corrupt output: ")
    print(corrupt_text[:200] + "..." if len(corrupt_text) > 200 else corrupt_text)
    print("\nPatched output: ")
    print(patched_text[:200] + "..." if len(patched_text) > 200 else patched_text)
    
    # Save patched results if requested
    if save_path:
        patched_data = {
            'patched_text': patched_text,
            'clean_text': clean_text,
            'corrupt_text': corrupt_text,
            'patched_token_ids': patched_ids.cpu(),
            'patch_stats': patch_stats,
            'patch_info': {
                'patch_layers': patch_layers,
                'patch_tokens_range': patch_tokens_range,
                'clean_path': clean_path,
                'corrupt_path': corrupt_path
            }
        }
        
        try:
            # Add file extension if not provided
            if not save_path.endswith('.pt'):
                save_path = f"{save_path}.pt"
            
            # Make sure directory exists
            os.makedirs(os.path.dirname(save_path) if os.path.dirname(save_path) else '.', exist_ok=True)
            
            torch.save(patched_data, save_path)
            print(f"Patched results saved to {save_path}")
        except Exception as e:
            print(f"Error saving patched results: {e}")
    
    return patched_text, clean_text, corrupt_text, patched_ids

In [11]:
clean_prompt = "What is the capital of Finland?"
clean_tokenized_prompt, clean_formatted_prompt = tokenize_prompt(clean_prompt, tokenizer, device=DEVICE)


clean_residual_streams, clean_generated_text, clean_token_ids = get_residual_stream(model, 
                                                                    tokenizer, 
                                                                    clean_tokenized_prompt, 
                                                                    max_new_tokens=512, 
                                                                    layer_indices=None, 
                                                                    save_path="res-stream/clean-prompt-1-test.pt")

if DEVICE == "cuda":
    torch.cuda.empty_cache()

print(clean_generated_text)

Prompt: <｜begin▁of▁sentence｜><｜User｜>What is the capital of Finland?<｜Assistant｜><think>

Saving activations to temporary directory: /var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/residual_stream_vt6817ts
Capturing 33 layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
Processing input sequence...
Input sequence processed (13 tokens)


Generating tokens:   0%|          | 0/512 [00:00<?, ?it/s]

Generated text: <｜User｜>What is the capital of Finland?<｜Assistant｜><think>\nOkay, so I need to figure out the ca...
Input tokens: 13, New tokens: 172, Total tokens: 185


/var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/ipykernel_40665/3106342554.py:223: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  activation = torch.load(file_path)


Residual stream data saved to res-stream/clean-prompt-1-test.pt 
-----

<｜User｜>What is the capital of Finland?<｜Assistant｜><think>
Okay, so I need to figure out the capital of Finland. Hmm, I'm not entirely sure, but I think it's one of the major cities in Northern Europe. I remember hearing that Helsinki is a big city there. Let me try to recall. I think Helsinki is the capital, but I'm not 100% certain. Maybe I should think about other capitals in Nordic countries. Sweden's capital is Stockholm, Norway's is Oslo, Denmark's is Copenhagen, and I believe Finland's is Helsinki. Yeah, that sounds right. I think I've heard Helsinki mentioned in the context of being a major port city and a cultural hub. I don't think it's Turku or any other city. So, putting it all together, I'm pretty confident that the capital of Finland is Helsinki.
</think>

The capital of Finland is Helsinki.


/var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/ipykernel_40665/3106342554.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  activation = torch.load(file_path)


In [12]:
corrupted_prompt = "What is the capital of Belgium?"
corrupted_tokenized_prompt, corrupted_formatted_prompt = tokenize_prompt(corrupted_prompt, tokenizer, device=DEVICE)


corrupted_residual_streams, corrupted_generated_text, corrupted_token_ids = get_residual_stream(model, 
                                                                    tokenizer, 
                                                                    corrupted_tokenized_prompt, 
                                                                    max_new_tokens=512, 
                                                                    layer_indices=None, 
                                                                    save_path="res-stream/corrupted-prompt-1-test.pt")

print(corrupted_generated_text)

Prompt: <｜begin▁of▁sentence｜><｜User｜>What is the capital of Belgium?<｜Assistant｜><think>

Saving activations to temporary directory: /var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/residual_stream_g_kho4x6
Capturing 33 layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
Processing input sequence...
Input sequence processed (13 tokens)


Generating tokens:   0%|          | 0/512 [00:00<?, ?it/s]

Generated text: <｜User｜>What is the capital of Belgium?<｜Assistant｜><think>\nOkay, so I need to figure out the ca...
Input tokens: 13, New tokens: 395, Total tokens: 408


/var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/ipykernel_40665/3106342554.py:223: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  activation = torch.load(file_path)
/var/fol

Residual stream data saved to res-stream/corrupted-prompt-1-test.pt 
-----

<｜User｜>What is the capital of Belgium?<｜Assistant｜><think>
Okay, so I need to figure out the capital of Belgium. Hmm, I'm not entirely sure, but I think it's one of those countries in Europe. Let me try to recall. I remember that Belgium is in Western Europe, right? It's one of the Benelux countries, along with the Netherlands and Luxembourg. I think the capital is a city that's pretty well-known, but I can't remember exactly which one.

Wait, I think it's not Brussels, but I'm not 100% sure. No, wait, I think Brussels is the capital. Yeah, I've heard it mentioned in news and movies. But I'm a bit confused because I also remember something about the EU having a capital, and I think it's in Brussels. So maybe that's why it's the capital of Belgium as well.

Let me think about other capitals of neighboring countries. France's capital is Paris, Germany's is Berlin, the Netherlands is Amsterdam, and Luxembourg is 

In [13]:
patched_text, clean_text, corrupt_text, patched_ids = patch_res_stream(model, 
                                                                        tokenizer, 
                                                                        clean_path = "res-stream/clean-prompt-1-test.pt", 
                                                                        corrupt_path = "res-stream/corrupted-prompt-1-test.pt", 
                                                                        patch_layers = [0], 
                                                                        num_tokens_to_patch=64, 
                                                                        target_token="<think>", 
                                                                        save_path="res-stream/patched-prompt-1-test.pt", 
                                                                        device=None, 
                                                                        max_new_tokens=512)

if DEVICE == "cuda":
    torch.cuda.empty_cache()

print(patched_text)

Using device: cpu
Loading residual streams from res-stream/clean-prompt-1-test.pt and res-stream/corrupted-prompt-1-test.pt...
Clean text preview: <｜User｜>What is the capital of Finland?<｜Assistant｜><think>
Okay, so I need to figure out the capita...
Corrupt text preview: <｜User｜>What is the capital of Belgium?<｜Assistant｜><think>
Okay, so I need to figure out the capita...


/var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/ipykernel_40665/2934087460.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(load_path)


Found target token '<think>' (ID: 128013) at position 11
Will patch tokens from position 11 to 75 (64 tokens)
Will patch layers: [0]
Preloading corrupted residual streams to device...


Preloading layers:   0%|          | 0/1 [00:00<?, ?it/s]

Patching tokens:   0%|                                                       | 0/64 [00:00<?, ?it/s]


--- Patching Results ---
Patched layers: [0]
Patched token range: (11, 75)
Total tokens processed: 64
Total tokens patched: 64
Tokens patched per layer: {0: 64}
Generation length: 209 new tokens

Original clean output: 
<｜User｜>What is the capital of Finland?<｜Assistant｜><think>
Okay, so I need to figure out the capital of Finland. Hmm, I'm not entirely sure, but I think it's one of the major cities in Northern Europ...

Original corrupt output: 
<｜User｜>What is the capital of Belgium?<｜Assistant｜><think>
Okay, so I need to figure out the capital of Belgium. Hmm, I'm not entirely sure, but I think it's one of those countries in Europe. Let me ...

Patched output: 
<｜User｜>What is the capital of Finland?<｜Assistant｜><think>

Alright, so I need to figure out the capital of Finland. Hmm, I'm not entirely sure. but I think it's one of those countries in Western. Le...
Patched results saved to res-stream/patched-prompt-1-test.pt
<｜User｜>What is the capital of Finland?<｜Assistant｜><think>



In [14]:
print("corrupted text:")
print(corrupt_text)
print("\n---")
print("clean text:")
print(clean_text)
print("\n---")
print("patched text:")
print(patched_text)
print("\n---")
# print(patched_ids)


corrupted text:
<｜User｜>What is the capital of Belgium?<｜Assistant｜><think>
Okay, so I need to figure out the capital of Belgium. Hmm, I'm not entirely sure, but I think it's one of those countries in Europe. Let me try to recall. I remember that Belgium is in Western Europe, right? It's one of the Benelux countries, along with the Netherlands and Luxembourg. I think the capital is a city that's pretty well-known, but I can't remember exactly which one.

Wait, I think it's not Brussels, but I'm not 100% sure. No, wait, I think Brussels is the capital. Yeah, I've heard it mentioned in news and movies. But I'm a bit confused because I also remember something about the EU having a capital, and I think it's in Brussels. So maybe that's why it's the capital of Belgium as well.

Let me think about other capitals of neighboring countries. France's capital is Paris, Germany's is Berlin, the Netherlands is Amsterdam, and Luxembourg is Luxembourg City. So, for Belgium, it's Brussels. I think tha

In [16]:
res_streams, generated_text, token_ids = load_residual_stream("res-stream/corrupted-prompt-1-test.pt")

/var/folders/wk/sgrd2bsj1msgt6bs3kfgj9hw0000gn/T/ipykernel_40665/2934087460.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(load_path)
